In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import deepchem as dc
from pubchempy import get_cids, get_compounds

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

In [ ]:
def display_df(df):
    display(HTML(df.to_html()))
    return None

In [ ]:
class CollinearColumnRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold, col_regex=None, exclude_cols=None):
        """
        :param threshold: float in [0, 1], if two columns have correlation greater than threshold
                          one of them will be removed
        :param col_regex: str, regular expression to select columns
        """
        self._threshold = threshold
        self._col_regex = col_regex
        if exclude_cols is None:
            self._exclude_cols = []
        else:
            self._exclude_cols = exclude_cols
    
    def _collinear_columns(self, df, threshold):
        if self._col_regex is None:
            df_sel = df.select_dtypes(["number", "bool"])
        else:
            df_sel = df.filter(regex=self._col_regex)
            df_sel = df_sel.select_dtypes(["number", "bool"])
        
        df_sel = df_sel.astype("float32")
        
        all_cols = df_sel.columns.to_list()
        all_cols = [col for col in all_cols if col not in self._exclude_cols]
        df_sel = df_sel[all_cols]
        ncols = len(all_cols)
        
        corr_mat = df_sel.corr().abs()
        self._corr_mat = corr_mat
        collin_cols = []
        for i in range(ncols-1):
            col_i = all_cols[i]
            if col_i in collin_cols:
                continue
            
            for j in range(i + 1, ncols):
                col_j = all_cols[j]
                if col_j in collin_cols:
                    continue
                
                corr = corr_mat.loc[col_i, col_j]
                if corr > threshold:
                    collin_cols.append(col_j)
        
        collin_cols = list(set(collin_cols))
        return collin_cols
    
    
    def fit(self, df):
        self._collin_cols = self._collinear_columns(df, self._threshold)
        return self
    
    def transform(self, df):
        all_cols = df.columns.to_list()
        nonexist_cols = [col for col in self._collin_cols if col not in all_cols]
        if len(nonexist_cols) > 0:
            print("WARNING: These collinear cols to be droped do not exist in df:", nonexist_cols)
            
        droped_col = [col for col in self._collin_cols if col in all_cols]
        print("Number of columns droped due to collinearity:", len(droped_col))
        return df.drop(droped_col, axis="columns")

In [ ]:
class NumImputer(BaseEstimator, TransformerMixin):
    def __init__(self, method="mean", exclude_cols=None):
        self._method = method
        if exclude_cols is None:
            self._exclude_cols = []
        else:
            self._exclude_cols = exclude_cols
    
    def fit(self, df_train):
        num_cols = df_train.select_dtypes(["number"]).columns.to_list()
        num_cols = [col for col in num_cols if col not in self._exclude_cols]
        
        self._train_cols = df_train.columns.to_list()
        
        self._impute_values = {}
        for col in num_cols:
            self._impute_values[col] = df_train[col].agg(self._method)
        return self
    
    def transform(self, df):
        df = df.copy()
        cols = df.columns.to_list()
        assert set(cols) == set(self._train_cols), "Do not have the same set of cols as train"
        
        for col, val in self._impute_values.items():
            if df[col].isnull().sum() > 0:
                df[col] = df[col].fillna(val)
        
        # align columns
        df = df[self._train_cols]
        return df
    

class CatImputer(BaseEstimator, TransformerMixin):
    def __init__(self, val="MISSING"):
        self._val = val
    
    def fit(self, df_train):
        cat_cols = df_train.select_dtypes(["object", "category", "bool"]).columns.to_list()
        self._train_cols = df_train.columns.to_list()
        
        self._impute_values = {}
        for col in cat_cols:
            self._impute_values[col] = self._val
        return self
    
    def transform(self, df):
        df = df.copy()
        cols = df.columns.to_list()
        assert set(cols) == set(self._train_cols), "Do not have the same set of cols as train"
        
        for col, val in self._impute_values.items():
            if df[col].isnull().sum() > 0:
                df[col] = df[col].astype("object").fillna(val).astype("category")
                
        # align columns
        df = df[self._train_cols]
        return df

In [ ]:
class Standardizer(BaseEstimator, TransformerMixin):
    def __init__(self, exclude_cols=None, to_array=False):
        if exclude_cols is None:
            self._exclude_cols = []
        else:
            self._exclude_cols = exclude_cols
            
        self._to_array = to_array
        
    def fit(self, df_train):
        num_cols = df_train.select_dtypes(["number"]).columns.to_list()
        num_cols = [col for col in num_cols if col not in self._exclude_cols]
        
        self._mean = {col: df_train[col].mean() for col in num_cols}
        self._std = {col: df_train[col].std() for col in num_cols}
        return self
    
    def transform(self, df):
        for col in self._mean:
            if self._std[col] > 0:
                df[col] = (df[col] - self._mean[col]) / self._std[col]
                df[col] = df[col].astype("float32")
            else:
                print("WARNING: " + col + " has zero std.")
                df[col] = df[col] - self._mean[col]
                df[col] = df[col].astype("float32")
                
        if self._to_array:
            return df.values.astype(np.float32)
        else:
            return df

In [ ]:
assert False

# pdY

In [ ]:
assert False

df_ache = pd.read_excel("data/raw/AchE.xlsx", sheet_name="ki_data_clean")

df_ache["new_id"] = np.arange(df_ache.shape[0])
df_ache["new_id"] = "_" + df_ache["new_id"].astype(str)
df_ache = df_ache.rename(columns={"Ligand SMILES": "smiles"})

print("df_ache", df_ache.shape)
display_df(df_ache.head())

df_ache["is_lower_bound"] = df_ache["Ki (nM)"].astype(str).apply(lambda s: s.startswith(">")).astype(int)
print("Number lower bound", df_ache["is_lower_bound"].sum())
df_ache["is_upper_bound"] = df_ache["Ki (nM)"].astype(str).apply(lambda s: s.startswith("<")).astype(int)
print("Number upper bound", df_ache["is_upper_bound"].sum())

# for lower bound cases we will multiply by 1.1
df_ache["ki_clean"] = df_ache["Ki (nM)"].astype(str).apply(lambda s: s.replace(">", "").replace("<", "")).astype(float)
df_ache.loc[df_ache["is_lower_bound"] == 1, ["ki_clean"]] *= 1.1
df_ache.loc[df_ache["is_upper_bound"] == 1, ["ki_clean"]] *= 0.9
print("df_ache", df_ache.shape)

df_ache["code"] = "labeled"
RT = 0.593
df_ache["dG"] = RT * np.log(df_ache["ki_clean"]*1e-09)

df_ache["smiles_len"] = df_ache["smiles"].apply(lambda x: len(x))

print("df_ache", df_ache.shape)
display_df(df_ache[["new_id", "smiles", "dG", "code", "smiles_len"]].head())

# dont need to remove long smiles
print("smile len", df_ache["smiles_len"].min(), df_ache["smiles_len"].max())


# remove samples which deviate too much from mean
df01 = df_ache.groupby(["smiles"], as_index=False).agg({"new_id": "count", "dG": ["min", "max", "mean"]})
df01.columns = ["smiles", "count", "dG_min", "dG_max", "dG_mean"]
df01["diff"] = df01["dG_max"] - df01["dG_min"]
print("df01", df01.shape)
display_df(df01.head())

df_ache = df_ache.merge(df01[["smiles", "dG_mean", "count"]], how="left", on="smiles")
df_ache["abs_dG_diff"] =  (df_ache["dG"] - df_ache["dG_mean"]).abs()
print("df_ache", df_ache.shape)
display_df(df_ache.head())

df_ache = df_ache[df_ache["abs_dG_diff"] <= 2.5]
print("df_ache", df_ache.shape)
display_df(df_ache.head())


df_ache.to_csv("data/process/ache_clean.csv", index=False)

# train/test

use the same test set as before

In [ ]:
assert False

# tvt before
tvt_bef = pd.read_csv("/home/hai/Compound_Screen/AchE_ML/data/process/pdY.csv")
tvt_bef = tvt_bef.loc[tvt_bef["code"]=="labeled_pubchem", ["smiles", "train_test"]]
print("tvt_bef", tvt_bef.shape, tvt_bef["smiles"].nunique())
display_df(tvt_bef.head())

df_tvt = pd.read_csv("data/process/ache_clean.csv")
print("df_tvt", df_tvt.shape)
df_tvt = df_tvt[["smiles"]].drop_duplicates()
print("df_tvt", df_tvt.shape)
display_df(df_tvt.head())

df_tvt = df_tvt.merge(tvt_bef, how="left", on="smiles")
print("df_tvt", df_tvt.shape)
display_df(df_tvt.head())

n_extra = df_tvt["train_test"].isnull().sum()
ntest2 = 200
tvt_extra = ["test"]*ntest2 + ["train"]*(n_extra - ntest2)
np.random.seed(42)
np.random.shuffle(tvt_extra)
df_tvt.loc[df_tvt["train_test"].isnull(), "train_test"] = tvt_extra
print("df_tvt", df_tvt.shape)
display_df(df_tvt.head())

df_tvt.to_csv("data/process/tvt.csv", index=False)

## labeled pdY for regression

In [ ]:
assert False

pdY = pd.read_csv("data/process/ache_clean.csv")
pdY = pdY[["new_id", "smiles", "code", "smiles_len", "dG"]]
print("pdY", pdY.shape)
display_df(pdY.head())

df_tvt = pd.read_csv("data/process/tvt.csv")
print("df_tvt", df_tvt.shape)
display_df(df_tvt.head())

pdY = pdY.merge(df_tvt, how="left", on="smiles")
print("pdY", pdY.shape)
display_df(pdY.head())

pdY.to_csv("data/process/pdY_labeled_reg.csv", index=False)

In [ ]:
pd.read_csv("data/process/pdY_labeled_reg.csv").head()

## NCI

In [ ]:
assert False

df_nci = pd.read_csv("data/raw/nci.smi", sep="\s+", header=None, dtype={0:str, 1: str, 2:str})
print(df_nci.shape)
df_nci.columns = ["smiles", "source", "id"]
df_nci["code"] = "nci"
df_nci["new_id"] = np.arange(df_nci.shape[0])
df_nci["new_id"] = "_" + df_nci["new_id"].astype(str)

df_nci["smiles_len"] = df_nci["smiles"].apply(lambda x: len(x))
df_nci["dG"] = np.nan

print(df_nci.shape)
display_df(df_nci.head())

df_nci.to_csv("data/process/nci_clean.csv", index=False)

In [ ]:
assert True

pdY = pd.read_csv("data/process/nci_clean.csv")
pdY = pdY[["new_id", "smiles", "code", "smiles_len", "dG"]]
pdY["train_test"] = "pred"
print("pdY", pdY.shape)
display_df(pdY.head())
pdY.to_csv("data/process/pdY_nci.csv", index=False)

In [ ]:
df_nci = pd.read_csv("data/raw/nci.smi", sep="\s+", header=None, dtype={0:str, 1: str, 2:str})
print(df_nci.shape)
display_df(df_nci.head())

# pdX

## Extract RDKitDescriptors

### labeled set

In [ ]:
assert False

pdY_labeled = pd.read_csv("data/process/pdY_labeled_reg.csv")
print("pdY_labeled", pdY_labeled.shape)
display_df(pdY_labeled.head())

rdkit_featurizer = dc.feat.RDKitDescriptors()
X = rdkit_featurizer(pdY_labeled["smiles"])

X1 = []
for y in X:
    if y.shape[0] > 0:
        X1.append(y.tolist())
    else:
        y = [np.nan]*200
        X1.append(y)
X1 = np.array(X1)

X2 = pd.DataFrame(X1, columns=rdkit_featurizer.descriptors)
X2["new_id"] = pdY_labeled["new_id"]
X2["smiles"] = pdY_labeled["smiles"]
X2["dG"] = pdY_labeled["dG"]
X2["code"] = pdY_labeled["code"]
X2["train_test"] = pdY_labeled["train_test"]
X2["smiles_len"] = pdY_labeled["smiles_len"]
display_df(X2.head())
if True:
    X2.to_csv("data/process/pdXY_labeled_rdkit_descriptors_200ft.csv", index=False)

## nci

In [ ]:
assert False

pdY_nci = pd.read_csv("data/process/pdY_nci.csv")
print("pdY_nci", pdY_nci.shape)
display_df(pdY_nci.head())

rdkit_featurizer = dc.feat.RDKitDescriptors()
X = rdkit_featurizer(pdY_nci["smiles"])

X1 = []
for y in X:
    if y.shape[0] > 0:
        X1.append(y.tolist())
    else:
        y = [np.nan]*200
        X1.append(y)
X1 = np.array(X1)

X2 = pd.DataFrame(X1, columns=rdkit_featurizer.descriptors)
X2["new_id"] = pdY_nci["new_id"]
X2["smiles"] = pdY_nci["smiles"]
X2["dG"] = pdY_nci["dG"]
X2["code"] = pdY_nci["code"]
X2["train_test"] = pdY_nci["train_test"]
X2["smiles_len"] = pdY_nci["smiles_len"]
display_df(X2.head())
if True:
    X2.to_csv("data/process/pdXY_nci_rdkit_descriptors_200ft.csv", index=False)
    
# there are 52 compunds whose smiles not matching

## Remove mostly zero columns

In [ ]:
assert True

pdXY_labeled = pd.read_csv("data/process/pdXY_labeled_rdkit_descriptors_200ft.csv")
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

PDY_COLS = ["new_id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY_labeled.columns if col not in PDY_COLS]

pdXY_train = pdXY_labeled[pdXY_labeled["train_test"] == "train"].copy()

mostly_zero_cols = []
for col in PDX_COLS:
    zero_rate = (pdXY_train[col] == 0).mean()
    if zero_rate > 0.95:
        print("{}    {}".format(col, zero_rate))
        mostly_zero_cols.append(col)

print("mostly_zero_cols", len(mostly_zero_cols))
print("there remain {}".format(len(PDX_COLS) - len(mostly_zero_cols)))

pdXY_labeled = pdXY_labeled.drop(mostly_zero_cols, axis=1)
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

pdXY_labeled.to_csv("data/process/pdXY_labeled_rdkit_descriptors_135ft.csv", index=False)

## nci

In [ ]:
assert False

pdXY_labeled = pd.read_csv("data/process/pdXY_labeled_rdkit_descriptors_200ft.csv")
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

PDY_COLS = ["new_id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY_labeled.columns if col not in PDY_COLS]

pdXY_train = pdXY_labeled[pdXY_labeled["train_test"] == "train"].copy()

mostly_zero_cols = []
for col in PDX_COLS:
    zero_rate = (pdXY_train[col] == 0).mean()
    if zero_rate > 0.95:
        print("{}    {}".format(col, zero_rate))
        mostly_zero_cols.append(col)

print("mostly_zero_cols", len(mostly_zero_cols))
print("there remain {}".format(len(PDX_COLS) - len(mostly_zero_cols)))

pdXY_nci = pd.read_csv("data/process/pdXY_nci_rdkit_descriptors_200ft.csv")
print(pdXY_nci.shape)
pdXY_nci = pdXY_nci.drop(mostly_zero_cols, axis=1)
print(pdXY_nci.shape)
display_df(pdXY_nci.head())

pdXY_nci.to_csv("data/process/pdXY_nci_rdkit_descriptors_135ft.csv", index=False)

## Remove correlated columns

### labeled set

In [ ]:
assert True

pdXY_labeled = pd.read_csv("data/process/pdXY_labeled_rdkit_descriptors_135ft.csv")
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

PDY_COLS = ["new_id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY_labeled.columns if col not in PDY_COLS]
print("PDX_COLS", len(PDX_COLS))

pdXY_train = pdXY_labeled[pdXY_labeled["train_test"] == "train"].copy()
print("pdXY_train", pdXY_train.shape)

remover = CollinearColumnRemover(0.95, exclude_cols=PDY_COLS)
remover.fit(pdXY_train)

pdXY_labeled = remover.transform(pdXY_labeled)
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

pdXY_labeled.to_csv("data/process/pdXY_labeled_rdkit_descriptors_108ft.csv", index=False)

## nci

In [ ]:
assert False

pdXY_labeled = pd.read_csv("data/process/pdXY_labeled_rdkit_descriptors_135ft.csv")
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

PDY_COLS = ["new_id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY_labeled.columns if col not in PDY_COLS]
print("PDX_COLS", len(PDX_COLS))

pdXY_train = pdXY_labeled[pdXY_labeled["train_test"] == "train"].copy()
print("pdXY_train", pdXY_train.shape)

remover = CollinearColumnRemover(0.95, exclude_cols=PDY_COLS)
remover.fit(pdXY_train)

pdXY_nci = pd.read_csv("data/process/pdXY_nci_rdkit_descriptors_135ft.csv")
print(pdXY_nci.shape)
pdXY_nci = remover.transform(pdXY_nci)
print(pdXY_nci.shape)
display_df(pdXY_nci.head())

pdXY_nci.to_csv("data/process/pdXY_nci_rdkit_descriptors_108ft.csv", index=False)

## Impute missing values

### labeled set

In [ ]:
assert True

pdXY_labeled = pd.read_csv("data/process/pdXY_labeled_rdkit_descriptors_108ft.csv")
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

PDY_COLS = ["new_id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY_labeled.columns if col not in PDY_COLS]
print("PDX_COLS", len(PDX_COLS))

pdXY_train = pdXY_labeled[pdXY_labeled["train_test"] == "train"].copy()
print("pdXY_train", pdXY_train.shape)

imputer = NumImputer(method="median", exclude_cols=PDY_COLS)
imputer.fit(pdXY_train)

pdXY_labeled = imputer.transform(pdXY_labeled)
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

pdXY_labeled.to_csv("data/process/pdXY_labeled_rdkit_descriptors_108ft_imputed.csv", index=False)

## nci

In [ ]:
assert False

pdXY_labeled = pd.read_csv("data/process/pdXY_labeled_rdkit_descriptors_108ft.csv")
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

PDY_COLS = ["new_id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY_labeled.columns if col not in PDY_COLS]
print("PDX_COLS", len(PDX_COLS))

pdXY_train = pdXY_labeled[pdXY_labeled["train_test"] == "train"].copy()
print("pdXY_train", pdXY_train.shape)

imputer = NumImputer(method="median", exclude_cols=PDY_COLS)
imputer.fit(pdXY_train)

pdXY_nci = pd.read_csv("data/process/pdXY_nci_rdkit_descriptors_108ft.csv")
print(pdXY_nci.shape)
pdXY_nci = imputer.transform(pdXY_nci)
print(pdXY_nci.shape)
display_df(pdXY_nci.head())

pdXY_nci.to_csv("data/process/pdXY_nci_rdkit_descriptors_108ft_imputed.csv", index=False)

## Standardize

### labeled set

In [ ]:
assert True

pdXY_labeled = pd.read_csv("data/process/pdXY_labeled_rdkit_descriptors_108ft_imputed.csv")
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

PDY_COLS = ["new_id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY_labeled.columns if col not in PDY_COLS]
print("PDX_COLS", len(PDX_COLS))

pdXY_train = pdXY_labeled[pdXY_labeled["train_test"] == "train"].copy()
print("pdXY_train", pdXY_train.shape)

std = Standardizer(exclude_cols=PDY_COLS)
std.fit(pdXY_train)

pdXY_labeled = std.transform(pdXY_labeled)
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

pdXY_labeled.to_csv("data/process/pdXY_labeled_rdkit_descriptors_108ft_imputed_std.csv", index=False)

## nci

In [ ]:
assert True

pdXY_labeled = pd.read_csv("data/process/pdXY_labeled_rdkit_descriptors_108ft_imputed.csv")
print(pdXY_labeled.shape)
display_df(pdXY_labeled.head())

PDY_COLS = ["new_id", "smiles", "dG", "code", "train_test", "smiles_len"]
PDX_COLS = [col for col in pdXY_labeled.columns if col not in PDY_COLS]
print("PDX_COLS", len(PDX_COLS))

pdXY_train = pdXY_labeled[pdXY_labeled["train_test"] == "train"].copy()
print("pdXY_train", pdXY_train.shape)

std = Standardizer(exclude_cols=PDY_COLS)
std.fit(pdXY_train)

pdXY_nci = pd.read_csv("data/process/pdXY_nci_rdkit_descriptors_108ft_imputed.csv")
print(pdXY_nci.shape)
pdXY_nci = std.transform(pdXY_nci)
print(pdXY_nci.shape)
display_df(pdXY_nci.head())

pdXY_nci.to_csv("data/process/pdXY_nci_rdkit_descriptors_108ft_imputed_std.csv", index=False)